## Introduction
Brief of Denpasar Weather Data Visualization and also a time series analytics, this is not finished work.

I build this kernel based on **analyticsvidhya** website over [here](https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/) and [here](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/).

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


In [ ]:
usecols = ['dt_iso', 'temp', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 
           'clouds_all', 'weather_main', 'weather_description']
df = pd.read_csv('../input/openweatherdata-denpasar-1990-2020v0.1.csv', parse_dates=True, index_col='dt_iso', usecols=usecols)
df['date'] = df.index
df = df[['date', 'temp', 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 
           'clouds_all', 'weather_main', 'weather_description']]
df.info()

In [ ]:
df.head(5)

## 1. Data Visualization

### 1.1 Number of Weather Main

In [ ]:
print(df['weather_main'].value_counts())
plt.figure(1, figsize=(8, 6))
sns.countplot(y='weather_main', data=df)
plt.show()

### 1.2 Number of Weather Description

In [ ]:
print(df['weather_description'].value_counts())
plt.figure(1, figsize=(7,15))
sns.countplot(y='weather_description', data=df)
plt.show()

In [ ]:
# Function to resampling time series data
def data_resample(data, time):
    """
    data: Dataframe
    time: Resampling frequencies
    """
    if time == 'hourly':
        data = data.resample('H').mean() # hour
    elif time == 'daily':
        data = data.resample('D').mean() # day
    elif time == 'weekly':
        data = data.resample('W').mean() # week
    elif time == 'monthly':
        data = data.resample('M').mean() # month
    elif time == 'quarterly':
        data = data.resample('Q').mean() # quarter
    elif time == 'yearly':
        data = data.resample('A').mean() # year
    
    return data

In [ ]:
def line_plot(data, plot_kind, xlabel, title):
    plt.figure(1, figsize=(12, 5))
    data.plot(kind=plot_kind)
    plt.xlabel(xlabel)
    plt.title(title)
    plt.show()

### 1.3 Line Plot

In [ ]:
line_plot(data_resample(df.loc['1990':'2019']['temp'], 'yearly'), 'line', 'year', 'Yearly Temperature Data')
line_plot(data_resample(df.loc['1990':'2019']['temp'], 'monthly'), 'line', 'year', 'Montly Temperature Data')
line_plot(data_resample(df.loc['1990':'2019']['temp'], 'weekly'), 'line', 'year', 'Weekly Temperature Data')

### 1.4 Histogram

In [ ]:
from scipy import stats

index = 0
dt_col = ['temp', 'pressure', 'humidity', 'wind_speed']
label = ['Temperature', 'Pressure', 'Humidity', 'Wind Speed']
plt.figure(1, figsize=(15,9))
for subplot in range(221, 225):
    plt.subplot(subplot)
    sns.distplot(df[dt_col[index]], kde=False, fit=stats.gamma)
    plt.xlabel(label[index], fontweight='bold')
    index += 1
plt.tight_layout()    
plt.show()

In [ ]:
index = 0
weather_col = ['wind_deg', 'clouds_all']
plt.figure(1, figsize=(15,9))
for subplot in range(221, 223):
    plt.subplot(subplot)
    sns.distplot(df[weather_col[index]], kde=False, fit=stats.gamma)
    plt.title(weather_col[index], fontweight='bold')
    index += 1
    
plt.show()

### 1.5 Polarized Bar Plot

In [ ]:
dt = df['wind_deg'].value_counts()
dt = pd.DataFrame(dt)
dt = dt.reset_index()
dt = dt.rename(columns={'index': 'wind_deg', 'wind_deg': 'num'})

N = len(dt)
width = np.pi / 4 * np.random.rand(N)
colors = plt.cm.viridis(radii / 10.)

ax = plt.subplot(111, projection='polar')
ax.bar(x=dt['wind_deg'], height=dt['num'], width=width, bottom=0.0, color=colors, alpha=0.5)
plt.figure(1, figsize=(15,9))
plt.show()

## 2. Stationarity

‘Stationarity’ is one of the most important concepts you will come across when working with time series data. **A stationary series is one in which the properties – mean, variance and covariance, do not vary with time**.

In [ ]:
# Check for stationarity
plt.figure(1, figsize=(15,6))
df['temp'].plot() # plot hourly data for 20 years

### 2.1 Check the data stationarity visualy

In [ ]:
index = 0
resample_time = ['yearly', 'monthly', 'weekly', 'daily']
plt.figure(1, figsize=(15,9))
for subplot in range(221, 225):
    plt.subplot(subplot)
    dt = data_resample(df['temp'], resample_time[index])
    plt.plot(dt)
    plt.title(resample_time[index], fontweight='bold')
    index = index + 1
    
plt.show()

In [ ]:
df_temp = data_resample(df['temp'].loc['1990':'2019'], 'daily')
df_temp.head()

From the above figure, we can determined that the daily temperature data is more stationary than the other chart.

### 2.2 Check the data stationarity statistically

There are many methods to determined the stationarity of time series data. In this part we only used several of them.

**1. ADF (Augmented Dickey Fuller) Test**

The Dickey Fuller test is one of the most popular statistical tests. It can be used to determine the presence of unit root in the series, and hence help us understand if the series is stationary or not. The null and alternate hypothesis of this test are:

- **Null Hypothesis**: The series has a unit root (value of a =1)

- **Alternate Hypothesis**: The series has no unit root.

If we fail to reject the null hypothesis, we can say that the series is non-stationary. This means that the series can be linear or difference stationary (we will understand more about difference stationary in the next section).

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries):
    print('Results of Dickey-Fuller Test:')
    df_test = adfuller(timeseries, autolag='AIC')
    df_output = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for key, value in df_test[4].items():
        df_output['Critical Value (%s)' % key] = value
    print(df_output)
    
# Apply ADF Test
adf_test(df_temp)

**Test for stationarity**: 

If the `Test Statistic` is less than the `Critical Value`, we can reject the null hypothesis (aka the **series is stationary**). When the `Test Statistic` is greater than the `Critical value`, we fail to reject the null hypothesis (which means the **series is not stationary**).

**2. KPSS (Kwiatkowski-Phillips-Schmidt-Shin) Test**

KPSS is another test for checking the stationarity of a time series (slightly less popular than the Dickey Fuller test). The null and alternate hypothesis for the KPSS test are opposite that of the ADF test, which often creates confusion.

The authors of the KPSS test have defined the null hypothesis as the process is trend stationary, to an alternate hypothesis of a unit root series. We will understand the trend stationarity in detail in the next section. For now, let’s focus on the implementation and see the results of the KPSS test.

**Null Hypothesis**: The process is trend stationary.

**Alternate Hypothesis**: The series has a unit root (series is not stationary).

In [ ]:
from statsmodels.tsa.stattools import kpss

def kpss_test(timeseries):
    print('Results of KPSS Test:')
    kpss_test = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpss_test[0:3], index=['Test Statistic', 'p-value', 'Lag Used'])
    for key, value in kpss_test[3].items():
        kpss_output['Critical Value (%s)' % key] = value
        
    print(kpss_output)
    
# Apply KPSS Test
kpss_test(df_temp)

**Test for stationarity**: 

If the `Test Statistic` is greater than the `Critical value`, we reject the null hypothesis (**series is not stationary**). If the `Test Statistic` is less than the `Critical value`, if fail to reject the null hypothesis (**series is stationary**).

**3. Types of Stationarity**
Let us understand the different types of stationarities and how to interpret the results of the above tests.

 - **Strict Stationary**: A strict stationary series satisfies the mathematical definition of a stationary process. For a strict stationary series, the mean, variance and covariance are not the function of time. The aim is to convert a non-stationary series into a strict stationary series for making predictions.
 - **Trend Stationary**: A series that has no unit root but exhibits a trend is referred to as a trend stationary series. Once the trend is removed, the resulting series will be strict stationary. The KPSS test classifies a series as stationary on the absence of unit root. This means that the series can be strict stationary or trend stationary.
 - **Difference Stationary**: A time series that can be made strict stationary by differencing falls under difference stationary. ADF test is also known as a difference stationarity test.

It’s always better to apply both the tests, so that we are sure that the series is truly stationary. Let us look at the possible outcomes of applying these stationary tests.
 - **Case 1**: Both tests conclude that the series is not stationary -> series is not stationary
 - **Case 2**: Both tests conclude that the series is stationary -> series is stationary
 - **Case 3**: KPSS = stationary and ADF = not stationary  -> trend stationary, remove the trend to make series strict stationary
 - **Case 4**: KPSS = not stationary and ADF = stationary -> difference stationary, use differencing to make series stationary

If we used hourly data, Case 4 is more suitable.

**4. Making a Time Series Stationary**

**Differencing**
This method computer the difference of consecutive terms in the series. Differencing is typically perfomed to get rid of the varying mean.

In [ ]:
df_diff = df_temp - df_temp.shift(1)
df_temp['diff'] = df_diff
df_temp['diff'].dropna().plot()

In [ ]:
def test_stationarity(timeseries, info):
    """
    timeseries: pandas time series
    info: resampling information, (hourly, daily, weekly, monthly, quarterly, yearly)
    """
    
    # Determining rolling statistics
    windowsize = 24
    rolmean = timeseries.rolling(windowsize).mean()
    rolstd = timeseries.rolling(windowsize).mean()
    
    # Plot rolling statistics:
    orig = plt.plot(timeseries, color='yellow', label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation ' + info)
    plt.show(block=False)
    
    # Perform Dickey-Fuller Test:
    adf_test(timeseries)
    
    print('\n')
    
    # Perform KPSS Test:
    kpss_test(timeseries)

In [ ]:
df_temp = data_resample(df.loc['1990':'2019']['temp'], 'hourly')
test_stationarity(df_temp, 'Monthly')